In [ ]:
############################################
import os 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
#
import numpy as np
#
# https://docs.pymc.io/
import pymc3 as pm
import theano
import time
from pymc3.backends.base import merge_traces
#
print('numpy      Ver.', np.__version__)
print('matplotlib Ver.', mpl.__version__)
print('pymc3      Ver.', pm.__version__)
print('theano     Ver.', theano.__version__)
#
#
file_path = "pdf/"
#
if( True != os.path.isdir(file_path) ):
    print('making ', file_path )
    os.mkdir(file_path)
else:
    print(file_path, ' is exist.' )

In [ ]:
######################################
XRDdata = np.loadtxt('20190810SAGA-LS.txt')
#
XRD_x = XRDdata[:,0]
XRD_y = XRDdata[:,1]

In [ ]:
######################################
def Grf1(X, Y):
    #
    plt.rcParams['font.size'] = 16
    #
    fig = plt.figure( figsize=(6,6) )
    fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
    #
    ax1 = fig.add_subplot(1,1,1)
    ax1.set_xlabel( r'$R2\theta/\phi$')
    ax1.set_ylabel('Intensity')
    ax1.set_yscale('log')
    #
    ax1.plot(X, Y, color='black', \
             linewidth=2.0, \
             label=r'$\phi=110.12^\circ$' )
    #
    plt.legend()
    #
    return fig

In [ ]:
fig = Grf1( XRD_x, XRD_y )

In [ ]:
############################################
def TwoGauss(X, a1, mu1, sd1, \
              a2, mu2, sd2, ):
    y = a1*np.exp(-(X-mu1)*(X-mu1)/(2*sd1*sd1)) + \
        a2*np.exp(-(X-mu2)*(X-mu2)/(2*sd2*sd2)) 
    return y

In [ ]:
fit_y = TwoGauss( XRD_x, 0.8, 39.31, 0.5, 3E-2, 38.92, 0.5 )

In [ ]:
######################################
def Grf2(X, Y1, Y2):
    #
    plt.rcParams['font.size'] = 16
    #
    fig = plt.figure( figsize=(6,6) )
    fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
    #
    ax1 = fig.add_subplot(1,1,1)
    ax1.set_xlabel( r'$R2\theta/\phi$')
    ax1.set_ylabel('Intensity')
    ax1.set_ylim(1E-4,1)
    ax1.set_yscale('log')
    #
    ax1.plot(X, Y1, color='black', \
             linewidth=2.0, \
             label=r'$\phi=110.12^\circ$' )
    #
    ax1.plot(X, Y2, color='red', \
             linewidth=1.5, \
             label='fit' )
    #
    plt.legend()
    #
    return fig

In [ ]:
fit_y = TwoGauss( XRD_x, 0.8, 39.31, 0.1, 3E-2, 38.92, 0.1 )
fig = Grf2( XRD_x, XRD_y, fit_y )

In [ ]:
############################################
def TwoLorentz(X, a1, mu1, w1, \
              a2, mu2, w2, ):
    y = a1 * w1*w1 /( (X-mu1)*(X-mu1) + w1*w1 ) + \
        a2 * w2*w2 /( (X-mu2)*(X-mu2) + w2*w2 ) 
    return y

In [ ]:
fit_y = TwoLorentz( XRD_x, 0.75, 39.31, 0.03, 2.5E-2, 38.92, 0.04 )
fig = Grf2( XRD_x, XRD_y, fit_y )

In [ ]:
############################################
def pVoigt1(X, mu, xc, wL, wG):
    y = mu     * (2/np.pi) * wL /( 4*(X-xc)*(X-xc) + wL*wL ) + \
        (1-mu) * (np.sqrt(4*np.log(2)/np.pi)/wG) * np.exp( - 4*np.log(2) * (X-xc)*(X-xc)/(wG*wG) ) 
    return y
############################################
def TwopVoigt1(X, a1, mu1, xc1, wL1, wG1, \
                  a2, mu2, xc2, wL2, wG2 ):
    y = a1 * pVoigt1(X, mu1, xc1, wL1, wG1) + \
        a2 * pVoigt1(X, mu2, xc2, wL2, wG2) 
    return y

In [ ]:
fit_y = TwopVoigt1( XRD_x, 8.4e-2, 0.5, 39.31, 0.10, 0.15, 
                           3.8E-3, 0.4, 38.92, 0.10, 0.25 )
fig = Grf2( XRD_x, XRD_y, fit_y )

In [ ]:
############################################
def TwopVoigt1v1(X, a1, xc1, wG1, \
                    a2, xc2, wG2 ):
    y = a1 * pVoigt1(X, 0.5, xc1, 0.1, wG1) + \
        a2 * pVoigt1(X, 0.4, xc2, 0.1, wG2) 
    return y

In [ ]:
fit_y = TwopVoigt1v1( XRD_x, 8.4e-2, 39.31, 0.15, 
                             3.8E-3, 38.92, 0.25 )
fig = Grf2( XRD_x, XRD_y, fit_y )

In [ ]:
a1v1  = 8.4e-2
xc1v1 = 39.31
wG1v1 = 0.15
a2v1  = 3.8E-3
xc2v1 = 38.92
wG2v1 = 0.25 

In [ ]:
with pm.Model() as model:
    a1 = pm.Gamma('a1', alpha=5.0,beta=50.0 )
    Gamma_a1 = a1.random(size=10000)

plt.rcParams['font.size'] = 16
#
fig = plt.figure( figsize=(6,6) )
fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
#
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel( r'$a$')
ax1.set_ylabel('Intensity')
#
bin_vals, bins, patches = ax1.hist( Gamma_a1, bins=100, density=True, 
                                    color='gray', label=r'$Gamma_{a}$')
#
(y_bot, y_top) = ax1.set_ylim()
#
ax1.vlines( a1v1, y_bot, y_top, 
            color='red', linestyles='dashed', linewidth=1.0, 
            label=r'$a_{1}$')
#
# ax1.vlines( a2v1, y_bot, y_top, 
#             color='blue', linestyles='dashed', linewidth=1.0, 
#             label=r'$a_{2}$')

In [ ]:
with pm.Model() as model:
    a2 = pm.Gamma('a2', alpha=5.0,beta=1050.0 )
    Gamma_a2 = a2.random(size=10000)

plt.rcParams['font.size'] = 16
#
fig = plt.figure( figsize=(6,6) )
fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
#
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel( r'$a$')
ax1.set_ylabel('Intensity')
#
bin_vals, bins, patches = ax1.hist( Gamma_a2, bins=100, density=True, 
                                    color='gray', label=r'$Gamma_{a}$')
#
(y_bot, y_top) = ax1.set_ylim()
#
# ax1.vlines( a1v1, y_bot, y_top, 
#             color='red', linestyles='dashed', linewidth=1.0, 
#             label=r'$a_{1}$')
#
ax1.vlines( a2v1, y_bot, y_top, 
            color='blue', linestyles='dashed', linewidth=1.0, 
            label=r'$a_{2}$')

In [ ]:
with pm.Model() as model:
    xc1 = pm.Normal('xc1', mu=39.31, sd=0.2 )
    Normal_xc1 = xc1.random(size=10000)

plt.rcParams['font.size'] = 16
#
fig = plt.figure( figsize=(6,6) )
fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
#
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel( r'$xc$')
ax1.set_ylabel('Intensity')
#
bin_vals, bins, patches = ax1.hist( Normal_xc1, bins=100, # density=True, 
                                    color='gray', label=r'$Gamma_{a}$')
#
(y_bot, y_top) = ax1.set_ylim()
#
ax1.vlines( xc1v1, y_bot, y_top, 
             color='red', linestyles='dashed', linewidth=1.0, 
             label=r'$xc_{1}$')
#
ax1.vlines( xc2v1, y_bot, y_top, 
            color='blue', linestyles='dashed', linewidth=1.0, 
            label=r'$xc_{2}$')

In [ ]:
with pm.Model() as model:
    xc2 = pm.Normal('xc2', mu=38.92, sd=0.2 )
    Normal_xc2 = xc2.random(size=10000)

plt.rcParams['font.size'] = 16
#
fig = plt.figure( figsize=(6,6) )
fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
#
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel( r'$xc$')
ax1.set_ylabel('Intensity')
#
bin_vals, bins, patches = ax1.hist( Normal_xc2, bins=100, # density=True, 
                                    color='gray', label=r'$Gamma_{a}$')
#
(y_bot, y_top) = ax1.set_ylim()
#
ax1.vlines( xc1v1, y_bot, y_top, 
             color='red', linestyles='dashed', linewidth=1.0, 
             label=r'$xc_{1}$')
#
ax1.vlines( xc2v1, y_bot, y_top, 
            color='blue', linestyles='dashed', linewidth=1.0, 
            label=r'$xc_{2}$')

In [ ]:
with pm.Model() as model:
    wG = pm.Gamma('wG', alpha=5.0,beta=21.0 )
    Gamma_wG = wG.random(size=10000)

plt.rcParams['font.size'] = 16
#
fig = plt.figure( figsize=(6,6) )
fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
#
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel( r'$wG$')
ax1.set_ylabel('Intensity')
#
bin_vals, bins, patches = ax1.hist( Gamma_wG, bins=100, density=True, 
                                    color='gray', label=r'$Gamma_{a}$')
#
(y_bot, y_top) = ax1.set_ylim()
#
ax1.vlines( wG1v1, y_bot, y_top, 
            color='red', linestyles='dashed', linewidth=1.0, 
            label=r'$wG_{1}$')
#
ax1.vlines( wG2v1, y_bot, y_top, 
            color='blue', linestyles='dashed', linewidth=1.0, 
            label=r'$wG_{2}$')

In [ ]:
with pm.Model() as model:
    #
    a1 = pm.Gamma('a1', alpha=5.0,beta=50.0 )
    a2 = pm.Gamma('a2', alpha=5.0,beta=1050.0 )
    #
    xc1 = pm.Normal('xc1', mu=39.31, sd=0.2 )
    xc2 = pm.Normal('xc2', mu=38.92, sd=0.2 )
    #
    wG1 = pm.Gamma('wG1', alpha=5.0,beta=21.0 )
    wG2 = pm.Gamma('wG2', alpha=5.0,beta=21.0 )
    #
    rmsd = pm.Uniform('rmsd', lower=0, upper=0.1)
    #
    y = pm.Normal('y', \
                  mu = TwopVoigt1v1( XRD_x, \
                                     a1, xc1, wG1, \
                                     a2, xc2, wG2 ), \
                  sd=rmsd, observed=XRD_y )
    #
    trace = pm.sample(10000, tune=10000, chains=2)

In [ ]:
pm.traceplot(trace)

In [ ]:
pm.summary(trace)

In [ ]:
summary_a1 = pm.summary( trace, ['a1'])
summary_a2 = pm.summary( trace, ['a2'])
summary_xc1 = pm.summary( trace, ['xc1'])
summary_xc2 = pm.summary( trace, ['xc2'])
summary_wG1 = pm.summary( trace, ['wG1'])
summary_wG2 = pm.summary( trace, ['wG2'])
a1_mean = np.float( summary_a1['mean'])
a1_sd   = np.float( summary_a1['sd'])
a2_mean = np.float( summary_a2['mean'])
a2_sd   = np.float( summary_a2['sd'])
xc1_mean = np.float( summary_xc1['mean'])
xc1_sd   = np.float( summary_xc1['sd'])
xc2_mean = np.float( summary_xc2['mean'])
xc2_sd   = np.float( summary_xc2['sd'])
wG1_mean = np.float( summary_wG1['mean'])
wG1_sd   = np.float( summary_wG1['sd'])
wG2_mean = np.float( summary_wG2['mean'])
wG2_sd   = np.float( summary_wG2['sd'])
print( 'a1  =  %.5f +- %.5f' % (a1_mean, a1_sd) )
print( 'xc1 = %.5f +- %.5f' % (xc1_mean, xc1_sd) )
print( 'wG1 =  %.5f +- %.5f' % (wG1_mean, wG1_sd) )
print( 'a2  =  %.5f +- %.5f' % (a2_mean, a2_sd) )
print( 'xc2 = %.5f +- %.5f' % (xc2_mean, xc2_sd) )
print( 'wG2 =  %.5f +- %.5f' % (wG2_mean, wG2_sd) )

In [ ]:
fit_y = TwopVoigt1v1( XRD_x, a1_mean, xc1_mean, wG1_mean, 
                             a2_mean, xc2_mean, wG2_mean )
fig = Grf2( XRD_x, XRD_y, fit_y )

In [ ]:
######################################
def Grf3(X, Y1, Y2, Y3):
    #
    plt.rcParams['font.size'] = 16
    #
    fig = plt.figure( figsize=(6,6) )
    fig.subplots_adjust( \
            left=0.15, right=0.98, 
            top=0.93,  bottom=0.1 )
    #
    ax1 = fig.add_subplot(1,1,1)
    ax1.set_xlabel( r'$R2\theta/\phi$')
    ax1.set_ylabel('Intensity')
    ax1.set_ylim(1E-4,1)
    ax1.set_yscale('log')
    #
    ax1.plot(X, Y1, color='black', \
             linewidth=2.0, \
             label=r'$\phi=110.12^\circ$' )
    #
    ax1.plot(X, Y2, color='red',  linewidth=1.5, label='peak1')
    ax1.plot(X, Y3, color='blue', linewidth=1.5, label='peak2')
    #
    plt.legend()
    #
    return fig

In [ ]:
fit_y1 = a1_mean * pVoigt1(XRD_x, 0.5, xc1_mean, 0.1, wG1_mean)
fit_y2 = a2_mean * pVoigt1(XRD_x, 0.4, xc2_mean, 0.1, wG2_mean)
fig = Grf3( XRD_x, XRD_y, fit_y1, fit_y2 )

In [ ]:
csv = np.zeros( (XRD_x.size, 4), dtype='float' )
csv[:,0] = XRD_x
csv[:,1] = fit_y1
csv[:,2] = fit_y2
csv[:,3] = fit_y

In [ ]:
np.savetxt('pymc-fit.txt', csv, delimiter=',')

In [ ]:
for samples in trace.get_values('a1', combine=False):
    print(samples)

In [ ]:
trace.get_values('a1', combine=False)[0]

In [ ]:
csv_sample_c1 = np.zeros( (10000, 6), dtype='float' )
csv_sample_c1[:,0] = trace.get_values('xc1', combine=False)[0]
csv_sample_c1[:,1] = trace.get_values('a1',  combine=False)[0]
csv_sample_c1[:,2] = trace.get_values('wG1',  combine=False)[0]
csv_sample_c1[:,3] = trace.get_values('xc2', combine=False)[0]
csv_sample_c1[:,4] = trace.get_values('a2',  combine=False)[0]
csv_sample_c1[:,5] = trace.get_values('wG2',  combine=False)[0]
np.savetxt('pymc-c1-sample.txt', csv_sample_c1, delimiter=',')

In [ ]:
csv_sample_c2 = np.zeros( (10000, 6), dtype='float' )
csv_sample_c2[:,0] = trace.get_values('xc1', combine=False)[1]
csv_sample_c2[:,1] = trace.get_values('a1',  combine=False)[1]
csv_sample_c2[:,2] = trace.get_values('wG1',  combine=False)[1]
csv_sample_c2[:,3] = trace.get_values('xc2', combine=False)[1]
csv_sample_c2[:,4] = trace.get_values('a2',  combine=False)[1]
csv_sample_c2[:,5] = trace.get_values('wG2',  combine=False)[1]
np.savetxt('pymc-c2-sample.txt', csv_sample_c2, delimiter=',')